# This notebook compiles the Fish Island Data, does QA/QC filtering, and adds custom variables

In [2]:
import numpy as np
import pandas as pd
import datetime
import Data_Processor as DP
import SunStatistics as SS

Root = 'C:/FishIsland_2017/'

Out = 'G:/My Drive/FishIsland_Outputs/'

FluxFolder='FluxResults/Outputs/'
TenHz_Path = Root+FluxFolder+'10Hz/eddypro_10Hz_full_output_2020-08-20T121146_exp.csv'
OneHz_Path = Root+FluxFolder+'1Hz/eddypro_1Hz_full_output_2020-08-20T183544_exp.csv'
Met_Path = Root+'MetStationData/CR1000_ClimateData.txt'
Soil_Path = Root+'SoilStationData/Soil_Data.csv'
Comp = DP.Compile([TenHz_Path,OneHz_Path],Met_Path,Soil_Path,#Daytime,Taglu_Data,NARRData,
                  Drop_Variables = ['daytime','none_flux','qc_none_flux','none_v-adv','none_molar_density',
                    'none_mole_fraction' ,'none_mixing_ratio','none_time_lag','none_spikes',
                    'none_var','w/none_cov','x_peak','x_offset','x_10%','x_30%','x_50%','x_70%','x_90%'])
Drop = [
['2017-07-10 10:00:00','2017-07-10 15:00:00'],
['2017-07-12 12:0:00','2017-07-12 14:00:00'],
['2017-08-01 10:00:00','2017-08-01 14:00:00'],
['2017-08-21 11:00:00','2017-08-21 14:00:00'],
['2017-09-12 07:30:00','2017-09-12 15:00:00']
       ]
Comp.Date_Drop(Drop,'2017-06-23 00:00:00')    

Comp.StorageCorrection()
Comp.QC_Check(2)
Rm1 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('QC Filter Removal:\n',Rm1)
Comp.Rain_Check([7,0])
Rm2 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('Rain Filter Removal:\n',Rm2-Rm1)
Comp.Signal_Check(20)
Rm3 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('RSSI Filter Removal:\n',Rm3-Rm2)
Comp.Wind_Filter(30,35+180)
Rm4 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('Wind Filter Removal:\n',Rm4-Rm3)
Comp.Spike_Removal(z_thresh=4)
Rm5 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('Spike Filter Removal:\n',Rm5-Rm4)
Comp.Ustar_Drop(Override=.1)
Rm6 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('u* Filter Removal:\n',Rm6-Rm5)
Comp.Wind_Bins(30)
Comp.CustomVars()

Data = Comp.Data.copy()
Data['Zm'] = 2.87
Data['Z0'] = Data['Zm']*np.exp(-.41*Data['wind_speed']/Data['u*'])

Data['Temp_15'] = Data[['Temp_15_1','Temp_15_2']].mean()

Data.index.name='datetime'

print('Final Sample Size')

print(Data[['fco2','fch4']].count())

QC Filter Removal:
 co2_flux    0.045883
ch4_flux    0.108497
dtype: float64
Rain Filter Removal:
 co2_flux    0.000000
ch4_flux    0.002179
dtype: float64
ch4_noSSFilter    2041
ch4_flux          2041
dtype: int64
RSSI Filter Removal:
 co2_flux    0.000000
ch4_flux    0.133333
dtype: float64
Wind Filter Removal:
 co2_flux    0.083281
ch4_flux    0.071895
dtype: float64
Spike Filter Removal:
 co2_flux    0.041169
ch4_flux    0.033115
dtype: float64
u* Filter Removal:
 co2_flux    0.043683
ch4_flux    0.023094
dtype: float64
Final Sample Size
fco2    2501
fch4    1441
dtype: int64


# Add day/night parameter and Save

In [3]:
LAT = 69.371182
LON = -134.880935
TZ = -6

Temp = Data[['DOY']].resample('5T').asfreq()
# Temp

D = Temp.index.floor('D').to_julian_date()#+.5
T = Temp.index.hour/24
A = np.ones(D.shape[0])

Zenith,Angle,Angle_Corr,Azimuth,Sunrise,Sunset=SS.SunStats(LAT*A,LON*A,D.values,T.values,TZ*A)
Temp['Sun_Angle'] = Angle_Corr
Temp = Temp.resample('30T').mean()
try:
    Data = Data.drop(columns=['Sun_Angle'])
except:
    pass
Data = Data.join(Temp[['Sun_Angle']])

Data['Daytime']=0
Data.loc[Data['Sun_Angle']>-0.5,'Daytime']=1
print('Bright Nights:',Data.loc[((Data.Daytime==0)&(Data.PPFD_Avg>10)),'PPFD_Avg'].count())
Data.loc[((Data.Daytime==0)&(Data.PPFD_Avg>10)),'Daytime']=1

File_Name = 'FI_Data_'+str(datetime.datetime.now()).split(' ')[0]+'.csv'
Data.to_csv(Out+File_Name)

C:\Users\wesle\Thesis_Code\SunStatistics.py:25: RuntimeWarning: invalid value encountered in arccos
  W2 = np.degrees(np.arccos(np.cos(np.radians(90.833))/(np.cos(np.radians(LAT))*np.cos(np.radians(T2)))-np.tan(np.radians(LAT))*np.tan(np.radians(T2)))) #HA Sunrise (deg)


Bright Nights: 4


In [5]:
F = Data.loc[Data.index.month==6]
F.groupby(F.index.hour)['PPFD_Avg'].mean()

datetime
0      115.043125
1       66.928125
2       38.233125
3       28.540938
4       33.716813
5       54.883125
6      103.795000
7      150.876875
8      250.512500
9      400.431250
10     512.637500
11     662.656250
12     911.731250
13    1086.106250
14    1135.831250
15    1163.437500
16    1153.275000
17    1064.250000
18     964.162500
19     837.068750
20     677.006250
21     501.031250
22     355.512500
23     243.660000
Name: PPFD_Avg, dtype: float64